## Let's create a golden test set with Synthetic Data Generation!

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import JSONLoader
import json

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["title"] = record.get("title", "")
    metadata["tags"] = record.get("tags", [])
    return metadata

loader = DirectoryLoader(
    path="data/designs",
    glob="**/*.json",
    loader_cls=JSONLoader,
    loader_kwargs={
        "jq_schema": ".",  # This will select the entire JSON object
        "content_key": None,  # We're not selecting a specific content key
        "metadata_func": metadata_func,
        "text_content": False
    }
)

documents = loader.load()

print(f"Loaded {len(documents)} documents")
if documents:
    print("\nExample document:")
    print(f"Content: {documents[0].page_content}")
    print(f"Metadata: {documents[0].metadata}")


Loaded 141 documents

Example document:
Content: {"id": "135", "url": "https://www.csszengarden.com/135", "css_url": "https://www.csszengarden.com/135/135.css", "description": "This design uses an elegant and traditional aesthetic, with a muted color palette featuring cream and terracotta tones. The use of a patterned background adds texture, while the vertical layout and serif typography provide a classic and sophisticated feel. The design balances text-heavy content with organized sections, making information easily accessible, and its ornate flourishes add a touch of refinement.", "categories": ["Traditional", "Elegant", "Text-Heavy", "Classic"], "visual_characteristics": ["Muted Color Palette", "Vertical Layout", "Serif Typography", "Textured Background", "Ornate Flourishes"]}
Metadata: {'source': '/Users/owner/Desktop/Projects/ai-maker-space/code/ImagineUI/src/data/designs/135/metadata.json', 'seq_num': 1, 'title': '', 'tags': []}
